In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
from server.model import load_model_main

import numpy as np
import torch
import guidance
from datetime import datetime
from langchain.embeddings import HuggingFaceEmbeddings
from server.generativeAgent import GenerativeAgent

# Load model

### Load embedding model

In [3]:
embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

### Load local LLM
### Please change these PATH

In [4]:
MODEL_PATH = '/home/quang/working/LLMs/oobabooga_linux/text-generation-webui/models/TheBloke_wizard-mega-13B-GPTQ'
CHECKPOINT_PATH = '/home/quang/working/LLMs/oobabooga_linux/text-generation-webui/models/TheBloke_wizard-mega-13B-GPTQ/wizard-mega-13B-GPTQ-4bit-128g.no-act.order.safetensors'
# MODEL_PATH = '/home/quang/working/LLMs/oobabooga_linux/text-generation-webui/models/TheBloke_Wizard-Vicuna-7B-Uncensored-GPTQ'
# CHECKPOINT_PATH = '/home/quang/working/LLMs/oobabooga_linux/text-generation-webui/models/TheBloke_Wizard-Vicuna-7B-Uncensored-GPTQ/Wizard-Vicuna-7B-Uncensored-GPTQ-4bit-128g.no-act-order.safetensors'

DEVICE = torch.device('cuda:0')
model, tokenizer = load_model_main(MODEL_PATH, CHECKPOINT_PATH, DEVICE)

Loading model ...
Done.


In [5]:
llama = guidance.llms.Transformers(model=model, tokenizer=tokenizer, device=DEVICE)
guidance.llm = llama

# Stat agent

In [6]:
# sam.silent = False
## Control the time so we can fast-forward
now = datetime.now()
new_time = now.replace(hour=7, minute=25)

In [7]:
description = "Sam is a Ph.D student, his major is CS;Sam likes computer;Sam lives with his friend, Bob;Sam's farther is a doctor;Sam has a dog, named Max"
sam = GenerativeAgent(guidance=guidance, 
                      name='Sam',
                      age=23, 
                      des=description, 
                      trails='funny, like football, play CSGO', 
                      embeddings_model=embeddings_model,
                      current_time=new_time)

Stop program ### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Sam has a dog, named Max 

### Response:
Rating: 1

### Add some memories

In [8]:
sam_observations = [
    "Sam wake up in the morning",
    "Sam feels tired because of playing games",
    "Sam has a assignment of AI course",
    "Sam see Max is sick",
    "Bob say hello to Sam",
    "Bob leave the room",
    "Sam say goodbye to Bob",
]

In [9]:
sam.add_memories(sam_observations)

Stop program ### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Sam say goodbye to Bob 

### Response:
Rating: 1

### Summary of Sam

In [10]:
summary = sam.get_summary(force_refresh=True)

Stop program ### Instruction:
 - Sam feels tired because of playing games
- Sam wake up in the morning
- Sam likes computer
- Sam is a Ph.D student, his major is CS
- Sam's farther is a doctor
- Sam lives with his friend, Bob
- Sam see Max is sick
- Sam say goodbye to Bob
- Sam has a dog, named Max
- Sam has a assignment of AI course 

### Input:
How would one describe Sam ’s feeling about his recent progress in life given the following statements?

### Response:
Based on the given statements, it is difficult to determine Sam's feeling about his recent progress in life. However, if we assume that Sam is satisfied with his progress, we can describe his feeling as content or fulfilled.

In [11]:
print(summary)

Name: Sam (age: 23)
Summary: Sam can be described as a Ph.D student who is passionate about computer science and enjoys playing games. He is also a responsible pet owner who takes care of his dog Max. Additionally, Sam is a hardworking student who has an assignment in his AI course. Sam is also a caring person who is concerned about his dog's health and has a supportive father who is a doctor. Finally, Sam is a sociable person who lives with his friend Bob and says goodbye to him before leaving.. Sam is a Ph.D student majoring in Computer Science. He wakes up in the morning and lives with his friend Bob. Sam has a dog named Max and is currently feeling tired due to playing games. Sam also has an assignment for his AI course.. it is difficult to determine Sam's feeling about his recent progress in life. However, if we assume that Sam is satisfied with his progress, we can describe his feeling as content or fulfilled.


### Let's update Sam's status

In [12]:
new_time = now.replace(hour=18, minute=15)
sam.set_current_time(new_time)

In [13]:
status = sam.update_status()

Stop program ### Instruction:
 Name: Sam (age: 23)
Summary: Sam can be described as a Ph.D student who is passionate about computer science and enjoys playing games. He is also a responsible pet owner who takes care of his dog Max. Additionally, Sam is a hardworking student who has an assignment in his AI course. Sam is also a caring person who is concerned about his dog's health and has a supportive father who is a doctor. Finally, Sam is a sociable person who lives with his friend Bob and says goodbye to him before leaving.. Sam is a Ph.D student majoring in Computer Science. He wakes up in the morning and lives with his friend Bob. Sam has a dog named Max and is currently feeling tired due to playing games. Sam also has an assignment for his AI course.. it is difficult to determine Sam's feeling about his recent progress in life. However, if we assume that Sam is satisfied with his progress, we can describe his feeling as content or fulfilled. 

Example for plan:
Here is Sam 's plan from now at 7:14:
[From 7:14 to 7:45]: Wake up and complete the morining routine
[From 7:45 to 8:35]: Eat breakfirst
[From 8:35 to 17:10]: Go to school and study
[From 17:10 to 22:30]: Play CSGO
[From 22:30 to 7:30]: Go to sleep

### Input:
Today is Tuesday June 06, 2023, 18:15 . Please make a plan today for Sam in broad strokes.

### Response:
Here is Sam 's plan from now at Tuesday June 06, 2023, 18:15 :
[From 18:15 to 19:00 ]: Get ready and leave for work 
 [From 19:00 to 20:00 ]: Attend meeting in office 
[From 20:00 to 21:00 ]: Start working on project 
[From 21:00 to 22:00 ]: Eat dinner

In [14]:
print(status)

Get ready and leave for work


### Add an observation, let's see how Sam react

In [15]:
bool_react, reaction, context = sam.react(observation='The dog bowl is empty', 
                                          observed_entity='Dog bowl', 
                                          entity_status='The dog bowl is empty')

Stop program ### Instruction:
 Name: Sam (age: 23)
Summary: Sam can be described as a Ph.D student who is passionate about computer science and enjoys playing games. He is also a responsible pet owner who takes care of his dog Max. Additionally, Sam is a hardworking student who has an assignment in his AI course. Sam is also a caring person who is concerned about his dog's health and has a supportive father who is a doctor. Finally, Sam is a sociable person who lives with his friend Bob and says goodbye to him before leaving.. Sam is a Ph.D student majoring in Computer Science. He wakes up in the morning and lives with his friend Bob. Sam has a dog named Max and is currently feeling tired due to playing games. Sam also has an assignment for his AI course.. it is difficult to determine Sam's feeling about his recent progress in life. However, if we assume that Sam is satisfied with his progress, we can describe his feeling as content or fulfilled. 

It is Tuesday June 06, 2023, 18:15 now.
 Sam 's status: Get ready and leave for work 
Observation: The dog bowl is empty 
Reaction: Sam should acknowledge the observation and take note of it. He should also ensure that Max is fed properly and given enough water to drink. 

Example for plan:
Here is Sam 's plan from now at 7:14:
[From 7:14 to 7:45]: Wake up and complete the morining routine
[From 7:45 to 8:35]: Eat breakfirst
[From 8:35 to 17:10]: Go to school and study
[From 17:10 to 22:30]: Play CSGO
[From 22:30 to 7:30]: Go to sleep

### Input:
It is Tuesday June 06, 2023, 18:15 now. Please make a plan from now for Sam in broad strokes given his reaction.

### Response:
Here is Sam 's plan from now at Tuesday June 06, 2023, 18:15 :
[From 18:15 to 19:00 ]: Go to school (attending lecture in AI course) 
 [From 19:00 to 20:00 ]: Eat lunch (at school cafeteria) 
[From 20:00 to 21:00 ]: Complete homework assignments in AI course 
[From 21:00 to 22:00 ]: Play CSGO

In [16]:
print(f"{bool_react}\nReaction: {reaction}\nContext: {context}")

Yes
Reaction: Sam should acknowledge the observation and take note of it. He should also ensure that Max is fed properly and given enough water to drink.
Context: - Sam has a dog, named Max


### Add a new agent, Bob - Sam's roomate

In [17]:
description = "Bob is a Ph.D student; His major is Art; He likes reading; He lives with his friend, Sam; He likes playing Minecraft"
bob = GenerativeAgent(guidance=guidance, 
                      name='Bob', 
                      age=23, 
                      des=description, 
                      trails='like pizza, frinedly, lazy', 
                      embeddings_model=embeddings_model)

Stop program ### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: He likes playing Minecraft 

### Response:
Rating: 2

In [18]:
summary = bob.get_summary(force_refresh=True)

Stop program ### Instruction:
 - Bob is a Ph.D student
- He lives with his friend, Sam
- He likes reading
- He likes playing Minecraft
- His major is Art 

### Input:
How would one describe Bob ’s feeling about his recent progress in life given the following statements?

### Response:
Based on the given statements, it is difficult to determine Bob's feeling about his recent progress in life. However, if we assume that Bob is satisfied with his progress, we could describe him as feeling content or fulfilled. If he is not satisfied, we could describe him as feeling frustrated or disappointed. Without more information, it is impossible to provide a more specific description of Bob's feeling.

In [19]:
print(summary)

Name: Bob (age: 23)
Summary: Bob's core characteristics can be described as follows:. Bob is likely a Ph.D student who lives with his friend Sam, and his major is Art. He also enjoys playing Minecraft and reading. It is not specified what his current daily occupation is, but it is likely that he is a student and spends most of his time studying and working on his Ph.D.. it is difficult to determine Bob's feeling about his recent progress in life. However, if we assume that Bob is satisfied with his progress, we could describe him as feeling content or fulfilled. If he is not satisfied, we could describe him as feeling frustrated or disappointed. Without more information, it is impossible to provide a more specific description of Bob's feeling.


In [20]:
status = bob.update_status()

Stop program ### Instruction:
 Name: Bob (age: 23)
Summary: Bob's core characteristics can be described as follows:. Bob is likely a Ph.D student who lives with his friend Sam, and his major is Art. He also enjoys playing Minecraft and reading. It is not specified what his current daily occupation is, but it is likely that he is a student and spends most of his time studying and working on his Ph.D.. it is difficult to determine Bob's feeling about his recent progress in life. However, if we assume that Bob is satisfied with his progress, we could describe him as feeling content or fulfilled. If he is not satisfied, we could describe him as feeling frustrated or disappointed. Without more information, it is impossible to provide a more specific description of Bob's feeling. 

Example for plan:
Here is Bob 's plan from now at 7:14:
[From 7:14 to 7:45]: Wake up and complete the morining routine
[From 7:45 to 8:35]: Eat breakfirst
[From 8:35 to 17:10]: Go to school and study
[From 17:10 to 22:30]: Play CSGO
[From 22:30 to 7:30]: Go to sleep

### Input:
Today is Tuesday June 06, 2023, 23:43 . Please make a plan today for Bob in broad strokes.

### Response:
Here is Bob 's plan from now at Tuesday June 06, 2023, 23:43 :
[From 23:43 to 7:14 ]: Go to sleep 
 [From 7:14 to 7:45 ]: Wake up and complete the morning routine 
[From 7:45 to 8:35 ]: Eat breakfast 
[From 8:35 to 17:10 ]: Go to school and study

In [21]:
print(status)

Go to sleep


### Interact between two agents

In [22]:
bool_react, reaction, context = sam.react(observation='Bob come room with a new PC', 
                                          observed_entity=bob,
                                          entity_status='Bob is setting up his new PC')

Stop program ### Instruction:
 Name: Sam (age: 23)
Summary: Sam can be described as a Ph.D student who is passionate about computer science and enjoys playing games. He is also a responsible pet owner who takes care of his dog Max. Additionally, Sam is a hardworking student who has an assignment in his AI course. Sam is also a caring person who is concerned about his dog's health and has a supportive father who is a doctor. Finally, Sam is a sociable person who lives with his friend Bob and says goodbye to him before leaving.. Sam is a Ph.D student majoring in Computer Science. He wakes up in the morning and lives with his friend Bob. Sam has a dog named Max and is currently feeling tired due to playing games. Sam also has an assignment for his AI course.. it is difficult to determine Sam's feeling about his recent progress in life. However, if we assume that Sam is satisfied with his progress, we can describe his feeling as content or fulfilled. 

It is Tuesday June 06, 2023, 18:15 now.
 Sam 's status: Go to school (attending lecture in AI course) 
Observation: Bob come room with a new PC 
Reaction: Sam says "Thank you, Bob." and asks for more information about the PC. Sam also asks if there is anything he can do to help. Sam checks on Max and sees that he is feeling better. 

Example for plan:
Here is Sam 's plan from now at 7:14:
[From 7:14 to 7:45]: Wake up and complete the morining routine
[From 7:45 to 8:35]: Eat breakfirst
[From 8:35 to 17:10]: Go to school and study
[From 17:10 to 22:30]: Play CSGO
[From 22:30 to 7:30]: Go to sleep

### Input:
It is Tuesday June 06, 2023, 18:15 now. Please make a plan from now for Sam in broad strokes given his reaction.

### Response:
Here is Sam 's plan from now at Tuesday June 06, 2023, 18:15 :
[From 18:15 to 19:00 ]: Arrive at school, check announcements, take attendance 
 [From 19:00 to 19:30 ]: Attend AI lecture 
[From 19:30 to 20:00 ]: Take notes 
[From 20:00 to 20:30 ]: Complete homework

In [23]:
print(f"{bool_react}\nReaction: {reaction}\nContext: {context}")

Yes
Reaction: Sam says "Thank you, Bob." and asks for more information about the PC. Sam also asks if there is anything he can do to help. Sam checks on Max and sees that he is feeling better.
Context: Sam lives with his friend Bob, who is setting up his new PC. Sam says goodbye to Bob and Bob says hello to Sam. Sam is a Ph.D student, his major is CS, and he likes computers. Sam's father is a doctor. Sam wakes up in the morning and has a dog named Max. Bob leaves the room, and Sam sees that Max is sick. Sam has an assignment for his AI course, and he feels tired because of playing games.


### Check the dialogue

In [24]:
print(sam.dialogue_list[0])

Tuesday June 06, 2023, 18:15

Sam: Hey Bob, what's up with your new PC?
Bob: It is really fast and responsive.
Sam: Wow, I'm glad to hear that. And how is Max doing?
Bob: He's feeling better now, thankfully.
Sam: That's great. I'm glad to hear it. Do you need any help with anything?
Bob: No, everything's fine, thanks for asking though.


### Okay, let's add more memories

In [25]:
sam_observations = [
    "Sam call his farther to ask about work",
    "Sam try to finish his assignment",
    "Sam remember his friend ask him for playing game",
    "Sam wake up, feel tired",
    "Sam have breakfast with Bob",
    "Sam look for a intership",
    "The big concert will take place in his school today",    
    "Sam's bike is broken",
    "Sam go to school quickly",
    "Sam find a good internship",
    "Sam make a plan to prepare his father birthday"
]

In [26]:
sam.add_memories(sam_observations)

Stop program ### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: 3. Sam lives with his friend Bob and has a passion for computers. 

### Response:
Rating: 3

In [27]:
summary = sam.get_summary(force_refresh=True)
status = sam.update_status()

Stop program ### Instruction:
 - Sam wake up, feel tired
- Sam feels tired because of playing games
- 1. Sam is a Ph.D student who is preparing for his father's birthday and trying to finish his assignment.
- Sam wake up in the morning
- Sam make a plan to prepare his father birthday
- Sam likes computer
- Sam try to finish his assignment
- Sam is a Ph.D student, his major is CS
- Sam go to school quickly
- Sam call his farther to ask about work 

### Input:
How would one describe Sam ’s feeling about his recent progress in life given the following statements?

### Response:
Based on the given statements, it is difficult to determine Sam's feeling about his recent progress in life. The statements do not provide any information about Sam's recent progress or any specific accomplishments that he has achieved. It is possible that Sam is feeling satisfied with his progress, but without any specific information, it is impossible to accurately describe his feeling.

In [28]:
print(summary)
print()
print(status)

Name: Sam (age: 23)
Summary: Sam can be described as a responsible, hardworking, and dedicated individual. He is a Ph.D student who is committed to his studies and is determined to complete his assignment. He is also a computer science major who enjoys working with computers. Additionally, he is focused on finding a good internship and making plans to prepare for his father's birthday. Overall, Sam is a disciplined and organized individual who is committed to achieving his goals.. Sam is a Ph.D student who is preparing for his father's birthday and trying to finish his assignment. He is also looking for an internship and has called his father to ask about work. Additionally, he wakes up in the morning feeling tired and makes a plan to prepare for his father's birthday. Sam's father is a doctor and he lives with his friend, Bob.. it is difficult to determine Sam's feeling about his recent progress in life. The statements do not provide any information about Sam's recent progress or any 

### Interesting part, the interview

In [29]:
response = sam.interview('Friend', 'Give an introduction of yourself.')

Stop program ### Instruction:
 Name: Sam (age: 23)
Summary: Sam can be described as a responsible, hardworking, and dedicated individual. He is a Ph.D student who is committed to his studies and is determined to complete his assignment. He is also a computer science major who enjoys working with computers. Additionally, he is focused on finding a good internship and making plans to prepare for his father's birthday. Overall, Sam is a disciplined and organized individual who is committed to achieving his goals.. Sam is a Ph.D student who is preparing for his father's birthday and trying to finish his assignment. He is also looking for an internship and has called his father to ask about work. Additionally, he wakes up in the morning feeling tired and makes a plan to prepare for his father's birthday. Sam's father is a doctor and he lives with his friend, Bob.. it is difficult to determine Sam's feeling about his recent progress in life. The statements do not provide any information about Sam's recent progress or any specific accomplishments that he has achieved. It is possible that Sam is feeling satisfied with his progress, but without any specific information, it is impossible to accurately describe his feeling. 

It is Tuesday June 06, 2023, 18:15 .
 Sam 's status: Arrive at school, check announcements, take attendance 

Summary of relevant context from Sam 's memory:
 - Sam has a assignment of AI course
- Sam find a good internship
- Sam try to finish his assignment
- 1. Sam is a Ph.D student who is preparing for his father's birthday and trying to finish his assignment.
- Bob say hello to Sam
- Sam look for a intership
- Sam wake up in the morning
- Sam wake up, feel tired
- Sam go to school quickly
- Sam is a Ph.D student, his major is CS 

### Input:
The Friend say " Give an introduction of yourself. ". What should Sam response?

### Response:
Here is the response from Sam : " Hi, I'm Sam. I'm a Ph.D student in computer science and I'm currently preparing for my father's birthday. I'm also looking for an internship and trying to finish my assignment for my AI course. I'm excited about the prospect of finding a good internship and I'm committed to achieving my goals. "

In [30]:
print(response)

Hi, I'm Sam. I'm a Ph.D student in computer science and I'm currently preparing for my father's birthday. I'm also looking for an internship and trying to finish my assignment for my AI course. I'm excited about the prospect of finding a good internship and I'm committed to achieving my goals.


In [31]:
response = sam.interview('Friend', 'Who do you live with?')

Stop program ### Instruction:
 Name: Sam (age: 23)
Summary: Sam can be described as a responsible, hardworking, and dedicated individual. He is a Ph.D student who is committed to his studies and is determined to complete his assignment. He is also a computer science major who enjoys working with computers. Additionally, he is focused on finding a good internship and making plans to prepare for his father's birthday. Overall, Sam is a disciplined and organized individual who is committed to achieving his goals.. Sam is a Ph.D student who is preparing for his father's birthday and trying to finish his assignment. He is also looking for an internship and has called his father to ask about work. Additionally, he wakes up in the morning feeling tired and makes a plan to prepare for his father's birthday. Sam's father is a doctor and he lives with his friend, Bob.. it is difficult to determine Sam's feeling about his recent progress in life. The statements do not provide any information about Sam's recent progress or any specific accomplishments that he has achieved. It is possible that Sam is feeling satisfied with his progress, but without any specific information, it is impossible to accurately describe his feeling. 

It is Tuesday June 06, 2023, 18:15 .
 Sam 's status: Arrive at school, check announcements, take attendance 

Summary of relevant context from Sam 's memory:
 - Sam lives with his friend, Bob
- 3. Sam lives with his friend Bob and has a passion for computers.
- Bob leave the room
- Sam have breakfast with Bob
- Sam say goodbye to Bob
- Bob say hello to Sam
- Sam wake up in the morning
- Sam has a dog, named Max
- 2. Sam is also looking for an internship and has a dog named Max.
- Sam wake up, feel tired 

### Input:
The Friend say " Who do you live with? ". What should Sam response?

### Response:
Here is the response from Sam : " I live with my friend, Bob. "

In [32]:
print(response)

I live with my friend, Bob.


In [33]:
response = sam.interview('Friend', 'who is Max?')

Stop program ### Instruction:
 Name: Sam (age: 23)
Summary: Sam can be described as a responsible, hardworking, and dedicated individual. He is a Ph.D student who is committed to his studies and is determined to complete his assignment. He is also a computer science major who enjoys working with computers. Additionally, he is focused on finding a good internship and making plans to prepare for his father's birthday. Overall, Sam is a disciplined and organized individual who is committed to achieving his goals.. Sam is a Ph.D student who is preparing for his father's birthday and trying to finish his assignment. He is also looking for an internship and has called his father to ask about work. Additionally, he wakes up in the morning feeling tired and makes a plan to prepare for his father's birthday. Sam's father is a doctor and he lives with his friend, Bob.. it is difficult to determine Sam's feeling about his recent progress in life. The statements do not provide any information about Sam's recent progress or any specific accomplishments that he has achieved. It is possible that Sam is feeling satisfied with his progress, but without any specific information, it is impossible to accurately describe his feeling. 

It is Tuesday June 06, 2023, 18:15 .
 Sam 's status: Arrive at school, check announcements, take attendance 

Summary of relevant context from Sam 's memory:
 - Sam see Max is sick
- Sam has a dog, named Max
- 2. Sam is also looking for an internship and has a dog named Max.
- The big concert will take place in his school today
- Sam is a Ph.D student, his major is CS
- 3. Sam lives with his friend Bob and has a passion for computers.
- Sam likes computer
- Sam's farther is a doctor
- Sam find a good internship
- Sam try to finish his assignment 

### Input:
The Friend say " who is Max? ". What should Sam response?

### Response:
Here is the response from Sam : " Max is my dog. I love him a lot. "

In [34]:
print(response)

Max is my dog. I love him a lot.
